In [2]:
import pandas as pd

# Load DeepPrime output from a CSV file
# Replace this with your actual filename
csv_path = "C:\\Users\\user\\Desktop\\GRIT-Lab\\FANCY-PANCE\\data\\sfgfp\\dp\\selected.csv"

df = pd.read_csv(csv_path)
df.head()

,PE,ID,WT74_On,Edited74_On,PBSlen,RTlen,RT-PBSlen,Edit_pos,Edit_len,RHA_len,...,nGCcnt1,nGCcnt2,nGCcnt3,fGCcont1,fGCcont2,fGCcont3,MFE3,MFE4,DeepSpCas9_score,DeepPrime_score
0,PE4max-e,sfGFP_Y66H_pos196_sub_tat_CAT_BFP,AACTGCCGGTTCCTTGGCCGACTCTGGTAACGACGCTGACTTATGG...,xxxxxxxxxxTCCTTGGCCGACTCTGGTAACGACGCTGACTCATGG...,11,26,37,21,1,5,...,7,14,21,63.636364,53.846154,56.756757,-7.0,-4.6,38.576965,62.740948
1,PE4max-e,sfGFP_T203Y_pos607_sub_acg_TAT_YFP,AACCATATGATCGCGTTTCTCGTTCGGATCTTTAGACAGAACGCTT...,xxxxxxxxGATCGCGTTTCTCGTTCGGATCTTTAGACAGAACGCTT...,13,39,52,28,3,9,...,7,16,23,53.846154,41.025641,44.230769,-8.1,-0.9,15.049713,9.985682


In [3]:
def reverse_complement(seq):
    complement = str.maketrans("ATCGatcg", "TAGCtagc")
    return seq.translate(complement)[::-1]

In [4]:
# basic SpCas9 76nt scaffold
SCAFFOLD = "GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGC"

# TevopreQ1 motif (for 3' stability)
TEVOPREQ1 = "CGCGGTTCTATCTAGTTACGCGTTAAACCAACTAGAA"

In [5]:
def build_epegRNA(row):
    wt = row['WT74_On']
    edited = row['Edited74_On']
    pbs_len = int(row['PBSlen'])
    rtt_len = int(row['RTlen'])

    # Spacer: skip first 4 nt, take next 20
    spacer = wt[4:24]

    # Clean edited sequence
    edited_clean = edited.replace('x', '')

    # Extract PBS and RTT
    pbs = edited_clean[:pbs_len]
    rtt = edited_clean[pbs_len:pbs_len + rtt_len]

    # Reverse complements
    pbs_rc = reverse_complement(pbs)
    rtt_rc = reverse_complement(rtt)

    return pd.Series({
        'spacer': spacer,
        'scaffold': SCAFFOLD,
        'template': rtt_rc,
        'PBS': pbs_rc,
        'seq_motif': TEVOPREQ1
    })

In [6]:
# Apply to all rows
epegRNA_df = df.apply(build_epegRNA, axis=1)

In [7]:
# Combine with original if you want to keep identifiers
if 'IID' in df.columns:
    epegRNA_df['peg_id'] = df['IID']

# Save output
epegRNA_df.to_csv("C:\\Users\\user\\Desktop\\GRIT-Lab\\FANCY-PANCE\\data\\sfgfp\\dp\\pegLIT_input.csv", index=False)

# Show preview
epegRNA_df.head()

,spacer,scaffold,template,PBS,seq_motif
0,GCCGGTTCCTTGGCCGACTC,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,ACCATGAGTCAGCGTCGTTACCAGAG,TCGGCCAAGGA,GTTCTATCTAGTTACGCGTTAAACCAACTAGAA
1,ATATGATCGCGTTTCTCGTT,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...,TATCTGAGCTATCAAAGCGTTCTGTCTAAAGATCCGAAC,GAGAAACGCGATC,GTTCTATCTAGTTACGCGTTAAACCAACTAGAA
